# Shadows of Doubt Demo

### 0. Setup: Initialize the engine and model instance

In [1]:
# Initialize the Engine
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from rtai.agent.agent_manager import AgentManager
from rtai.agent.agent import Agent
from rtai.world.world import World
from rtai.world.clock import clock

cfg = YamlLoader.load("configs/rtai_neil.yaml")
timer = clock(cfg.expand('Clock'))
event_queue = Queue()

world = World(cfg.expand('World'), event_queue)

llm_client = LLMClient()
llm_client.initialize(cfg.expand('LLMClient'))

agent_mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client, world=world)
agent_mgr.initialize()

mark: Agent = agent_mgr.agents['Mark Turner'] # defense attorney
sarah: Agent = agent_mgr.agents['Sarah Reynolds'] # witness
emily: Agent = agent_mgr.agents['Emily Thorton'] # defendant

/opt/homebrew/Caskroom/miniforge/base/envs/embeddings_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
# Initialize the model
import guidance
from guidance import models
from guidance import gen, select

local_model_path = cfg.expand("LLMClient").get_value("local_model_path")
mistral = models.LlamaCpp(local_model_path, n_gpu_layers=-1, n_ctx=7000)

# initialize mythos as a scriptwriter
mythos = mistral + "You are a master Hollywood scriptwriter who specializes in legal dramas. Conduct each task as if you are that scriptwriter."

In [2]:
# utility functions
import json

def dump(a, filename):
    with open(filename, 'w') as f:
        json.dump(a, f)

def read_json(f_path):
    return json.loads(open(f_path).read())

# add the facts to agents' memories
def add_facts(agent, facts):
    for fact in facts:
        agent.l_mem.add_concept(fact)

def get_memory(agent):
    return list([node.summary() for node in agent.l_mem.id_to_node.values()])

def clean_dialogue(dialogue):
    cleaned = dialogue.split("\n")
    speakers = []
    utterances = []
    for line in cleaned:
        if line == '':
            continue
        else:
            split_line = line.split(":", maxsplit=1)
            try:
                speaker, utterance = split_line[0], split_line[1]
                # if speaker != 'Mark Turner' != 'Sarah Reynolds' != 'Emily Thorton':
                #     continue
                speakers.append(speaker)
                utterances.append(utterance)
            except:
                continue
    return speakers, utterances

def show_dialogue(speakers, utterances):
    for speaker, utterance in zip(speakers, utterances):
        print(speaker+": "+utterance)

def add_dialogue(speakers, utterances):
    # add a new space for the chat
    dialogue = []
    for speaker, utterance in zip(speakers, utterances):
        dialogue.append({'speaker': speaker, 'utterance': utterance})
    return dialogue

In [5]:
to_visualize = read_json('/Users/nyeung/Projects/mythos/WebAI/demo/actually_done.json')
to_visualize.keys()

dict_keys(['mark_inital_memory', 'mark_emily_dialogue', 'mark_reflection', 'mark_next_steps', 'mark_emily_dialogue_2', 'mark_new_memory'])

[' There was a man wearing a red tie and a suit who left the party right after an argument, which Sarah thought was alarming.',
 " Sarah can't remember the man's face but is willing to help Mark investigate further.",
 " This potential sighting could be an important lead in the investigation of Richard's poisoning."]

In [4]:
# the facts of the case
sarah_facts = ["Sarah witnessed a heated argument between Emily Thorton and Richard Thornton during the social gathering.",
"Sarah noticed Emily Thorton and Richard Thorton's strained relationship in the weeks leading up to the event.",
"Sarah is aware of financial difficulties in the Thornton marriage. Richard Thorton was considering a divorce partly due to their financial difficulties.",
"During the argument, Sarah overheard Emily expressing frustration about Richard's intention to file for divorce.",
"Sarah saw a man in suit and a red tie leave the party shortly after the argument.",]

mark_facts = [
    "Mark knows that Richard had business rivals who might have had motives to harm him.",
    "Mark thinks there is a potential alibi for Emily during the time of the poisoning, but has no idea about who it might be",
    "Mark knows that Richard Thorton was sometimes shady in buisness."
]

emily_facts = [
    "Emily is aware of the severe financial struggles within her marriage to Richard.",
    "Emily had heated arguments about money, and Richard was considering a divorce partly due to their financial difficulties.",
    "Emily received anonymous threatening letters in the weeks leading up to the incident.",
    "Emily was aware of Richard's involvement in a secret business deal that had caused friction in their personal and professional lives.",
    "The details of this deal were not fully disclosed to her, but she knew it played a role in their marriage difficulties.",
    "Emily can provide insight into her emotional state on the night of the incident.",
    "Emily recalls an unknown buisness person visiting their home on the night of the incident.",
]

In [5]:
# add the facts to the agents' memories
add_facts(sarah, sarah_facts)
add_facts(mark, mark_facts)
add_facts(emily, emily_facts)

In [6]:
to_visualize = {}
to_visualize['mark_inital_memory'] = get_memory(mark)
to_visualize[']

['Mark knows that Richard had business rivals who might have had motives to harm him.',
 'Mark thinks there is a potential alibi for Emily during the time of the poisoning, but has no idea about who it might be',
 'Mark knows that Richard Thorton was sometimes shady in buisness.']

### Generate the dialogue

In [7]:
dialogue_topic = "Mark asks Sarah about the party"
location = "On a walk outside Mark's office"

In [8]:
mark.l_mem.retriever.retrieve_context(dialogue_topic, 2)

recency score for Mark thinks there is a potential alibi for Emily during the time of the poisoning, but has no idea about who it might be is 10.0
recency score for Mark knows that Richard had business rivals who might have had motives to harm him. is 10.0
recency score for Mark knows that Richard Thorton was sometimes shady in buisness. is 10.0


'Mark thinks there is a potential alibi for Emily during the time of the poisoning, but has no idea about who it might be\nMark knows that Richard had business rivals who might have had motives to harm him.\n'

In [23]:
mythos.reset()

In [31]:
# function to generate dialogue
@guidance
def generate_dialogue(lm, agent1, agent2, location, dialogue_topic):
    agent1_name = agent1.get_name()
    agent2_name = agent2.get_name()
    agent1_persona = agent1.get_common_set_str() # + f" {agent1_name} MUST SPEAKS LIKE A LAWYER FROM THE SOUTH"
    agent2_persona = agent2.get_common_set_str() # + f" {agent2_name} MUST SPEAK LIKE A WITNESS WHO IS ARISTOCRATIC AND BRITISH"
    agent1_ctx = agent1.l_mem.retriever.retrieve_context(dialogue_topic, 1)
    agent2_ctx = agent2.l_mem.retriever.retrieve_context(dialogue_topic)
    prompt = f'''
    Generate a short dialogue between {agent1_name} and {agent2_name} in {location} about {dialogue_topic}.

    {agent2_name}'s persona: {agent2_persona}
    {agent2_name}'s context: {agent2_ctx}
    
    {agent1_name} persona: {agent1_persona}

    MAKE SURE THAT MARK ASKS  {agent2_name} ABOUT {agent1_ctx}
    
    Here is the short dialogue:
    {gen('dialogue', temperature=0.9, max_tokens=500)}"""
    '''
    lm += prompt
    return lm

mythos += generate_dialogue(mark, sarah, location, dialogue_topic)

In [32]:
# access the generated dialogue and reset the state of storyteller model
dialogue = mythos["dialogue"]
mythos.reset()

In [33]:
speakers, utterances = clean_dialogue(dialogue)
to_visualize['mark_emily_dialogue'] = add_dialogue(speakers=speakers, utterances=utterances)
to_visualize['mark_emily_dialogue']

[{'speaker': 'Mark Turner',
  'utterance': ' Hey Sarah, how are you doing? I heard it was a lovely evening at the party.'},
 {'speaker': 'Sarah Reynolds',
  'utterance': ' (nervously) Yes, it was nice. The food was great, and the company was wonderful. But... (pauses)'},
 {'speaker': 'Mark Turner', 'utterance': " But? What's on your mind, Sarah?"},
 {'speaker': 'Sarah Reynolds',
  'utterance': ' (hesitantly) Mark, I might have seen something that night...'},
 {'speaker': 'Mark Turner',
  'utterance': ' Something important? You know you can trust me with this.'},
 {'speaker': 'Sarah Reynolds',
  'utterance': ' (whispers) Yes, Mark. I saw a man leave the party right after the argument. He was wearing a red tie and a suit.'},
 {'speaker': 'Mark Turner',
  'utterance': ' A man in a red tie and a suit? Do you remember his face?'},
 {'speaker': 'Sarah Reynolds',
  'utterance': ' (shakes her head) No, not really. But I remember the feeling of alarm I had when I saw him leave.'},
 {'speaker': 

In [ ]:
mark.l_mem.retriever.retrieve_context(dialogue_topic)

In [ ]:
mark_ctx = mark.l_mem.retriever.retrieve_context(dialogue_topic)
mark_ctx

### Reflect on the dialogue

In [91]:
from guidance import select

@guidance
def reflect(lm, agent1, agent2, dialogue):
    agent1_name = agent1.get_name()
    agent2_name = agent2.get_name()
    agent1_ctx = agent1.l_mem.retriever.retrieve_context(dialogue)
    lm.set("agent1_ctx", agent1_ctx)
    lm += f'''
    Given this dialogue, what are the top 3 NEW takeaways for {agent1_name} from this dialogue?
    
    Here is the dialogue: {dialogue}
    
    OLD takeaways for {agent1_name}:
    {agent1_ctx}

    Here are the NEW takeaways: MAKE SURE TO STATE EACH TAKEAWAY AS MARK KNOWS...
    {gen(name="takeaways", temperature=1.0, max_tokens=300)}'''

    # next action
    # lm += f'''
    # Based off those three takwaways what should {agent1_name} do next?

    # {agent1_name}'s options are to THEORIZE: take time to theorize a timeline about what happened, CHAT: go chat with another witness to gather more information
    # {select(name="do_next", options=["THEORIZE", "CHAT"])} because {gen(name="reason", temperature=0.8, max_tokens=300)}'''
    # if lm["do_next"] == "THEORIZE":
    #     lm += f'''Q: What is {agent1_name}'s current theory about what happened?
    #     A: {gen(name="theory", temperature=0.8, max_tokens=300)}.'''
    # if lm["do_next"] == "CHAT":
    #     lm += f'''Q: Who should {agent1_name} chat with next to learn more information about the case? Prioritize witnesses who have not been interviewed yet.

    #     Think step by steps
    #     {gen(name="reasoning", temperature=0.8, max_tokens=300)}

    #     I will now select with who to chat next with:
    #     Who to chat next with: {select(name="next_chat", options=["Emily Thorton", "Emily Thorton"])}. Topic of the next chat: {gen(name="topic", stop=".", temperature=0.8, max_tokens=100)}.'''
    return lm

# prompt = f'''Who should Mark chat with next? {select(name="next_chat", options=["Emily Thorton", "Sarah"])}'''
# mythos_reasoning = mistral + "You are analyzing what Mark Turner learned from a chat with Sarah."
# mythos_reasoning += reflect(agent1=mark, agent2=sarah, dialogue=dialogue)
# mythos_reasoning += prompt

@guidance
def act(lm, takeaways):
    lm += f'''
    Based off these takeaways what should Mark do next?

    Takeaways: {takeaways}

    WAIT AROUND: wait around for more information to come in
    CHAT: chat with more witnesses to gather more infromation

    {select(name="next_action", options=['WAIT AROUND', 'CHAT'])}'''
    if lm["next_action"] == "CHAT":
        lm += f'''Who should Mark Chat with next? {select(name="next_chat", options=["Emily Thorton", "Sarah"])}. Mark should chat about {gen(name="topic", stop=".", temperature=0.8, max_tokens=20)}'''
    return lm

In [92]:
mythos.reset()

In [93]:
mythos += reflect(agent1=mark, agent2=sarah, dialogue=dialogue)

In [46]:
takeaways = mythos["takeaways"]

In [52]:
# Now we add the takeaways from the dialogue to Mark's memory
takeaways = mythos["takeaways"].split("\n")
out = []
# clean the takeaways
for takeaway in takeaways:
    if takeaway == '':
        continue
    else:
        cleaned_takeaway = takeaway.split(".", maxsplit=1)[1]
        if cleaned_takeaway == '':
            continue
        out.append(cleaned_takeaway)
        mark.l_mem.add_concept(cleaned_takeaway)

In [98]:
mythos.reset()

In [95]:
mythos += act(takeaways=out)

In [96]:
to_visualize['mark_next_steps'] = f"NEXT STEP: {mythos['next_action']} with {mythos['next_chat']} about {mythos['topic']}"
to_visualize['mark_next_steps']

'NEXT STEP: CHAT with Emily Thorton about the man in red tie sighting and ask if she and Richard had alibis for the night'

In [97]:
dump(to_visualize, 'nyeung_intermediate_results_final.json')

In [54]:
get_memory(mark)

['Mark knows that Richard had business rivals who might have had motives to harm him.',
 'Mark thinks there is a potential alibi for Emily during the time of the poisoning, but has no idea about who it might be',
 'Mark knows that Richard Thorton was sometimes shady in buisness.',
 ' There was a man wearing a red tie and a suit who left the party right after an argument, which Sarah thought was alarming.',
 " Sarah can't remember the man's face but is willing to help Mark investigate further.",
 " This potential sighting could be an important lead in the investigation of Richard's poisoning.",
 ' Mark now believes that there is a potential sighting of a man in a red tie and a suit who may have been involved in the poisoning of Richard.',
 " Sarah's description of this man is limited to his attire, and she doesn't remember his face or any other distinguishing features.",
 ' Mark recognizes the need to follow up on this lead by speaking with Emily and Richard about the sighting and their

In [56]:
to_visualize['mark_reflection'] = out

In [58]:
dump(to_visualize, 'intermediate_results.json')

In [102]:
mythos.reset()

### Chatting with Emily

In [103]:
mythos += generate_dialogue(mark, emily, location, dialogue_topic=to_visualize['mark_next_steps'])

In [104]:
# access the generated dialogue and reset the state of storyteller model
dialogue = mythos["dialogue"]
mythos.reset()

In [105]:
to_visualize['mark_emily_dialogue_2'] = add_dialogue(*clean_dialogue(dialogue))
to_visualize['mark_emily_dialogue_2']

[{'speaker': 'Mark Turner',
  'utterance': ' Emily, I need to follow up on something regarding that night. Did you or Richard see the man in the red tie around the house that evening?'},
 {'speaker': 'Emily Thorton',
  'utterance': " (pauses) I...I remember a man coming to our door, but I didn't get a good look at him. Richard dealt with it. What makes you ask?"},
 {'speaker': 'Mark Turner',
  'utterance': " A witness reported seeing a man matching that description near Richard's office that night. It could be a mere coincidence, but as your attorney, I must explore all leads. Could you, or Richard, provide an alibi for that night?"},
 {'speaker': 'Emily Thorton',
  'utterance': ' (nervously) We were at home, and we had dinner guests over. I remember a lot of conversations, laughter, and a few arguments. Richard and I were dealing with personal and financial issues at the time, and I can recall the details of the evening rather vividly. Would that suffice?'},
 {'speaker': 'Mark Turner'

In [108]:
to_visualize = get_memory(mark)

['Mark knows that Richard had business rivals who might have had motives to harm him.',
 'Mark thinks there is a potential alibi for Emily during the time of the poisoning, but has no idea about who it might be',
 'Mark knows that Richard Thorton was sometimes shady in buisness.',
 ' There was a man wearing a red tie and a suit who left the party right after an argument, which Sarah thought was alarming.',
 " Sarah can't remember the man's face but is willing to help Mark investigate further.",
 " This potential sighting could be an important lead in the investigation of Richard's poisoning.",
 ' Mark now believes that there is a potential sighting of a man in a red tie and a suit who may have been involved in the poisoning of Richard.',
 " Sarah's description of this man is limited to his attire, and she doesn't remember his face or any other distinguishing features.",
 ' Mark recognizes the need to follow up on this lead by speaking with Emily and Richard about the sighting and their

In [109]:
to_visualize = read_json('almost_done.json')